In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
import pingouin as pg
from statsmodels.stats.multitest import multipletests
from sklearn.utils import resample
import scikit_posthocs as sp
from itertools import combinations
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.multivariate.manova import MANOVA
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
df = pd.read_pickle("../../data/adhd-beliefs-pt/adhd-beliefs-pt-liwc-proportional.pkl")
features = df.columns[-64:].tolist()

In [ ]:
# Prepare data & groups
df['group'] = (
    df['sex'].map({'Feminino':'Female','Masculino':'Male'}).astype(str) + '_' +
    np.where(df['adhd_diagnosis']=="Sim, diagnosticado", 'ADHD',
        np.where(df['adhd_diagnosis'].isin(["Suspeito que tenho", "Estou em processo de diagnóstico"]), 'SuspectADHD', 'noADHD')
    )
)
groups = df['group'].unique().tolist()

In [ ]:
# number of participants in each group
group_counts = df['group'].value_counts()
print("Group counts:")
print(group_counts)

In [ ]:
# Permutation Welch ANOVA
def perm_anova_F(feat, group_col='group', n_perm=5000):
    obs = pg.welch_anova(dv=feat, between=group_col, data=df)
    f_obs = obs.at[0, 'F']
    perm_F = []
    for _ in range(n_perm):
        shuffled = np.random.permutation(df[group_col].values)
        perm_df = df.assign(__grp=shuffled)
        res = pg.welch_anova(dv=feat, between='__grp', data=perm_df)
        perm_F.append(res.at[0, 'F'])
    p_perm = (np.sum(np.array(perm_F) >= f_obs) + 1) / (n_perm + 1)
    return f_obs, p_perm

In [ ]:
# Omnibus tests per feature
omni_rows = []
for feat in features:
    # a) Covariate‐adjusted ANOVA
    null = smf.ols(f"{feat} ~ age + C(education) + word_count", data=df).fit()
    full = smf.ols(f"{feat} ~ C(group) + age + C(education) + word_count", data=df).fit()
    an3 = anova_lm(full, typ=3)
    F_adj = an3.loc['C(group)', 'F']
    p_raw = an3.loc['C(group)', 'PR(>F)']
    # b) Permutation
    _, p_perm = perm_anova_F(feat)
    # c) Kruskal–Wallis
    samples = [g[feat].dropna() for _, g in df.groupby('group')]
    H, p_kw = stats.kruskal(*samples)
    # d) Bayes factor via BIC
    bf10 = np.exp((null.bic - full.bic) / 2)
    omni_rows.append({
        'feature': feat, 'F_adj': F_adj, 'p_adj_raw': p_raw,
        'p_perm': p_perm, 'H': H, 'p_kw': p_kw, 'BF10': bf10
    })
omni_df = pd.DataFrame(omni_rows)
omni_df['p_adj_fdr'] = multipletests(omni_df['p_adj_raw'], method='fdr_bh')[1]
omni_df['p_kw_fdr']  = multipletests(omni_df['p_kw'],      method='fdr_bh')[1]
omni_df = omni_df.sort_values('p_adj_raw')
# Keep top 5 for post‐hoc
top5 = omni_df.head(5)['feature'].tolist()

In [ ]:
# Omnibus summary (top 5 features)
print("=== Omnibus ANOVA (top 5) ===")
print(omni_df[['feature','F_adj','p_adj_raw','p_adj_fdr','p_perm','BF10']].head(5).to_markdown(index=False))

In [ ]:
# Residualize features
resid_df = df.copy()
for feat in features:
    m = smf.ols(f"{feat} ~ age + C(education) + word_count", data=df).fit()
    resid_df[feat] = m.resid

In [ ]:
# Games–Howell contrasts & effect sizes for top features
def hedges_g(x, y):
    nx, ny = len(x), len(y)
    sx, sy = x.std(ddof=1), y.std(ddof=1)
    s_p = np.sqrt(((nx-1)*sx**2 + (ny-1)*sy**2)/(nx+ny-2))
    g = (x.mean() - y.mean())/s_p
    return g*(1 - 3/(4*(nx+ny)-9))

def bootstrap_ci(x, y, statfunc, n_boot=2000, ci=95):
    vals = []
    # precompute arrays
    x = np.asarray(x)
    y = np.asarray(y)
    nx, ny = len(x), len(y)
    for _ in range(n_boot):
        idx_x = np.random.randint(0, nx, size=nx)    # bootstrap indices for x
        idx_y = np.random.randint(0, ny, size=ny)    # bootstrap indices for y
        bx = x[idx_x]
        by = y[idx_y]
        vals.append(statfunc(bx, by))
    lower = np.percentile(vals, (100-ci)/2)
    upper = np.percentile(vals, 100-(100-ci)/2)
    return lower, upper

In [ ]:
pairs = list(combinations(groups, 2))
posthoc = {}
for feat in top5:
    gh = pg.pairwise_gameshowell(dv=feat, between='group', data=resid_df)
    gh['p_fdr'] = multipletests(gh['pval'], method='fdr_bh')[1]
    es = []
    for A, B in pairs:
        x = resid_df.loc[resid_df.group == A, feat].dropna().values
        y = resid_df.loc[resid_df.group == B, feat].dropna().values
        g = hedges_g(x, y)
        lo, hi = bootstrap_ci(x, y, hedges_g)
        es.append({'A': A, 'B': B, 'g': g, 'ci_lower': lo, 'ci_upper': hi})
    posthoc[feat] = {'games_howell': gh, 'effect_sizes': pd.DataFrame(es)}

In [ ]:
# Post-hoc Games-Howell and effect sizes
for feat in top5:
    print(f"=== Post-hoc for {feat} ===")
    print("Games-Howell:")
    print(posthoc[feat]['games_howell'][['A','B','T','pval','p_fdr']].round(3).to_markdown(index=False))
    print("Effect sizes (Hedges' g + 95% CI):")
    print(posthoc[feat]['effect_sizes'].round(3).to_markdown(index=False))

In [ ]:
# Dunn’s test on residuals
dunn_tests = {}
for feat in top5:
    dunn = sp.posthoc_dunn(resid_df, val_col=feat, group_col='group', p_adjust='fdr_bh')
    dunn_tests[feat] = dunn

In [ ]:
# Dunn's tests
print("=== Dunn's tests ===")
for feat in top5:
    print(f"Dunn's for {feat}:")
    print(dunn_tests[feat].round(3).to_markdown())

In [ ]:
# Assumption checks
def assumption_checks(feature):
    samples = [df[df.group == g][feature].dropna() for g in groups]
    levene_p = stats.levene(*samples, center='median').pvalue
    model = smf.ols(f"{feature} ~ C(group) + age + C(education) + word_count", data=df).fit()
    shapiro_p = stats.shapiro(model.resid)[1]
    infl = OLSInfluence(model)
    cooks = infl.cooks_distance[0]
    n_high = np.sum(cooks > 4/len(df))
    return {'levene_p': levene_p, 'shapiro_p': shapiro_p, 'n_high_cook': n_high}
assump = {feat: assumption_checks(feat) for feat in top5}

In [ ]:
print("=== Assumption Checks (top features) ===")
for feat, res in assump.items():
    print(f"{feat}: Levene p={res['levene_p']:.3f}, Shapiro p={res['shapiro_p']:.3f}, High Cook's={res['n_high_cook']}")

In [ ]:
# MANOVA
# Use formula interface to treat 'group' as categorical automatically
formula = ' + '.join(features) + ' ~ group'
# statsmodels MANOVA supports from_formula
man = MANOVA.from_formula(formula, data=df)
manova_res = man.mv_test()

In [ ]:
# MANOVA result summary
print("=== MANOVA ===")
print(manova_res)

In [ ]:
# Canonical variates & discriminant analysis
# Prepare data for LDA: drop missing and covariates
X = df[features].dropna()
y = df.loc[X.index, 'group']

# Fit LDA with up to min(n_classes-1, n_features) components
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X, y)

# Append to DataFrame for plotting
df_lda = pd.DataFrame(X_lda, columns=['LD1','LD2'], index=X.index)
df_lda['group'] = y

# Scatter plot of first two canonical axes
plt.figure(figsize=(8,6))
for grp in df_lda['group'].unique():
    subset = df_lda[df_lda['group']==grp]
    plt.scatter(subset['LD1'], subset['LD2'], label=grp, alpha=0.7)
plt.legend()
plt.title('LDA: First two linear discriminants')
plt.xlabel('LD1')
plt.ylabel('LD2')
plt.tight_layout()
plt.show()

In [ ]:
# Feature loadings on discriminants
# sklearn LDA uses `scalings_` for linear discriminants (n_features x n_components)
scalings = pd.DataFrame(lda.scalings_[:, :2], index=features, columns=['LD1','LD2'])
# Sort to find top contributors by absolute weight
top_ld1 = scalings['LD1'].abs().sort_values(ascending=False).head(10)
top_ld2 = scalings['LD2'].abs().sort_values(ascending=False).head(10)

In [ ]:
print("=== Feature scalings on discriminants ===")
print('Top 10 features loading on LD1:')
print(scalings.loc[top_ld1.index, 'LD1'])
print('Top 10 features loading on LD2:')
print(scalings.loc[top_ld2.index, 'LD2'])

In [ ]:
explained = lda.explained_variance_ratio_
print(f"Explained variance by LD1: {explained[0]:.2%}")
print(f"Explained variance by LD2: {explained[1]:.2%}")

In [ ]:
# PCA
pca = PCA(n_components=5)
df_pca = df.dropna(subset=features)
pcs = pca.fit_transform(df_pca[features])
for i in range(5): df_pca[f'PC{i+1}'] = pcs[:, i]
pc_anovas = {f'PC{i+1}': pg.welch_anova(dv=f'PC{i+1}', between='group', data=df_pca) for i in range(5)}
pc1_anova = pc_anovas['PC1']
pc2_anova = pc_anovas['PC2']
pc3_anova = pc_anovas['PC3']
pc4_anova = pc_anovas['PC4']
pc5_anova = pc_anovas['PC5']

In [ ]:
# PCA -> PC1 ANOVA
print("=== PCA PC1 ANOVA ===")
print(pc1_anova)
print("=== PCA PC2 ANOVA ===")
print(pc2_anova)
print("=== PCA PC3 ANOVA ===")
print(pc3_anova)
print("=== PCA PC4 ANOVA ===")
print(pc4_anova)
print("=== PCA PC5 ANOVA ===")
print(pc5_anova)

In [ ]:
explained = pca.explained_variance_ratio_
for i, var in enumerate(explained, 1):
    print(f"PC{i}: {var:.2%} of total variance")

In [ ]:
loadings = pd.DataFrame(pca.components_.T, index=features,
                        columns=[f"PC{i}" for i in range(1,6)])
for i in range(1, 6):
    print(f"Top contributors to PC{i}:")
    print(loadings[f'PC{i}'].abs().sort_values(ascending=False).head(10))

In [ ]:
sns.boxplot(x='group', y='PC1', data=df_pca)
plt.xticks(rotation=45)
plt.title("PC1 scores by group")
plt.show()

In [ ]:
merged = df_pca.join(df_lda[['LD1']])
r, p = pearsonr(merged['PC1'], merged['LD1'])
print(f"PC1 vs. LD1: r={r:.2f}, p={p:.3f}")

In [ ]:
df_numeric = df.copy()
print(df_numeric['education'].unique())
edu_map = {'Ensino secundário': 1, 'Licenciatura': 2, 'Pós-Graduação': 3, 'Mestrado': 4, 'Doutoramento': 5}
df_numeric['education'] = df_numeric['education'].map(edu_map).astype(int)
print(df_numeric['education'].unique())

In [ ]:
# Rank ANCOVA
df_rank = df_numeric.copy()
rank_ancova = {}
for feat in features:
    # rank-transform feature
    df_rank[f'{feat}_rank'] = df_rank[feat].rank()
    res = pg.ancova(data=df_rank, dv=f'{feat}_rank', covar=['age','education','word_count'], between='group')
    rank_ancova[feat] = res

In [ ]:
# Rank-ANCOVA on top features
print("=== Rank-based ANCOVA (top features) ===")
for feat in top5:
    print(f"{feat}:")
    print(rank_ancova[feat])

In [ ]:
# Residuals Visualizations
def plot_residuals(feat):
    mdl = smf.ols(f"{feat} ~ age + C(education) + word_count", data=df).fit()
    resid = mdl.resid
    plt.figure()
    sns.violinplot(x=df['group'], y=resid, inner='quartile')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

for feat in top5[:3]:
    plt.figure(figsize=(8, 5))
    mdl = smf.ols(f"{feat} ~ age + C(education) + word_count", data=df).fit()
    resid = mdl.resid
    sns.violinplot(x=df['group'], y=resid, inner='quartile', hue=df['group'], palette='Set2', legend=False)
    sns.stripplot(x=df['group'], y=resid, color='k', alpha=0.3, jitter=0.2, size=2)
    plt.title(f"Residuals of {feat} by group")
    plt.xlabel("")
    plt.ylabel(f"{feat} (residual)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Bootstrap ANCOVA F CI
def bootstrap_ancova(feat, n_boot=1000):
    Fs = []
    df_bsa = df_numeric.copy()
    for _ in range(n_boot):
        samp = df_bsa.sample(frac=1, replace=True)
        mod = smf.ols(f"{feat} ~ C(group) + age + education + word_count", data=samp).fit()
        Fs.append(anova_lm(mod, typ=3).loc['C(group)', 'F'])
    return np.percentile(Fs, [2.5, 97.5])
ci_boot = {feat: bootstrap_ancova(feat) for feat in top5}

In [ ]:
print("=== Bootstrap ANCOVA F 95% CI ===")
for feat, ci in ci_boot.items():
    print(f"{feat}: {ci[0]:.2f} to {ci[1]:.2f}")